In [ ]:
import os
from glob import glob

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
% matplotlib inline
# Define some variables for later

PATH = os.getcwd()
LOG_DIR = PATH + '/logOld'

# Import images and labels from CSV

In [ ]:
labels = pd.read_csv('AVL-Images.csv')  # reading the csv file
print(labels.head())  # printing first five rows of the file

train_image = []
files = glob('../images/*.jpg')
for file in files:
    img = image.load_img(file, target_size=(256, 256, 3))
    img = image.img_to_array(img)
    img = img / 255
    img = img[..., ::-1]
    train_image.append(img)
X = np.array(train_image)
y = np.array(labels.loc[:, ['Albert', 'Eline', 'Erik', 'Kevin', 'Marcel']])
y.shape

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=30, test_size=0.1)

#Exact size of train_X to fill with data to extract in the following for loop.
fillable_X = np.zeros((264, 256, 256, 3), dtype=np.float32)

# Generate data and apply modification

In [ ]:
datagen_X = np.concatenate((train_X, fillable_X), axis=3)
datagen = image.ImageDataGenerator(featurewise_center=True, rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
datagen.fit(datagen_X)
X_batches = datagen_X
y_batches = train_y

In [ ]:
epochs = 6

for e in range(epochs):
    print('Epoch', e)
    batches = 0
    batch_size = 6
    for X_batch, y_batch in datagen.flow(datagen_X, train_y, batch_size=batch_size):
        X_batches = np.concatenate((X_batches, X_batch), axis=0)
        y_batches = np.concatenate((y_batches, y_batch), axis=0)
        batches += 1
        if batches >= len(datagen_X) / batch_size:
            break

#Imagegenerator adds another dimension that we don't need so we remove it.
train_X_new = X_batches[:, :, :, :3]
train_X_new.shape

In [ ]:

modification = image.save_img("modification1.jpg", train_X_new[458][..., ::-1])
plt.imshow(train_X_new[458][..., ::-1])

In [ ]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(5), activation="relu", input_shape=(256, 256, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=32, kernel_size=(5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu', name='intermediate'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
earlystopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=2, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)


In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=1, write_images=True)
hist = model.fit(train_X, train_y, shuffle=True, epochs=100, validation_data=(test_X, test_y), callbacks=[tensorboard_callback, earlystopping],
                 class_weight={0: 0.5653846154, 1: 0.5495327103, 2: 0.4983050847, 3: 0.5345454545, 4: 0.6461538462})
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.show()

In [ ]:
model.save('modelold')

# Predict classes for an image

In [ ]:
img = image.load_img('../erik.jpg', target_size=(256, 256, 3))
img = image.img_to_array(img)
img = img / 255
classes = np.array(labels.columns[2:])
proba = model.predict(img.reshape(1, 256, 256, 3))
top_3 = np.argsort(proba[0])[:-4:-1]
for i in range(3):
    print("{}".format(classes[top_3[i]]) + " ({:.3})".format(proba[0][top_3[i]]))
plt.imshow(img)

# Featurevector file

In [ ]:
def img_features(image_file_name):
    image_features = np.zeros((1, 5))
    im = image.load_img(image_file_name, target_size=(256, 256, 3))
    im = image.img_to_array(im)
    im = im / 255
    #last layer of model
    image_features[0, :] = model.predict(im.reshape(1, 256, 256, 3))[0]
    return image_features

In [ ]:
PATH = os.getcwd()
data_path = '../images'
data_dir_list = os.listdir(data_path)

image_features_list = []

for img in sorted(data_dir_list):
    if (img == '.DS_Store'):
        continue
    image_features = img_features(data_path + '/' + img)
    image_features_list.append(image_features)

image_features_arr = np.asarray(image_features_list)
image_features_arr = np.rollaxis(image_features_arr, 1, 0)
image_features_arr = image_features_arr[0, :, :]

np.savetxt('feature_vectors_400_samples.txt', image_features_arr)

In [ ]:
feature_vectors = np.loadtxt('feature_vectors_400_samples.txt')
features = tf.Variable(feature_vectors, name='features')

# Setting up the Tensorboard projector

In [ ]:
import os
import cv2
import shutil
from tensorboard.plugins import projector
import tensorflow as tf

In [ ]:
def plot_to_projector(
        x,
        feature_vector,
        y,
        class_names,
        log_dir="logold",
        meta_file="metadata.tsv",
):
    assert x.ndim == 4  # (BATCH, H, W, C)

    if os.path.isdir(log_dir):
        shutil.rmtree(log_dir)

    # Create a new clean fresh folder :)
    os.mkdir(log_dir)

    SPRITES_FILE = os.path.join(log_dir, "sprites.png")
    sprite_x = x * 255
    sprite = create_sprite(sprite_x)
    cv2.imwrite(SPRITES_FILE, sprite)

    # Generate label names
    labels = []
    for i in range(int(y.shape[0])):
        labels.append([])
        for j in range(len(y[i])):
            if (y[i][j] != 0):
                labels[i].append(class_names[j])

    with open(os.path.join(log_dir, meta_file), "w") as f:
        for label in labels:
            f.write("{}\n".format(label))

    if feature_vector.ndim != 2:
        print(
            "NOTE: Feature vector is not of form (BATCH, FEATURES)"
            " reshaping to try and get it to this form!"
        )
        feature_vector = tf.reshape(feature_vector, [feature_vector.shape[0], -1])
    print(feature_vector.shape)

    feature_vector = tf.Variable(feature_vector)
    checkpoint = tf.train.Checkpoint(embedding=feature_vector)
    checkpoint.save(os.path.join(log_dir, "embeddings.ckpt"))

    # Set up config
    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
    embedding.metadata_path = meta_file
    embedding.sprite.image_path = "sprites.png"
    embedding.sprite.single_image_dim.extend((x.shape[1], x.shape[2]))
    projector.visualize_embeddings(log_dir, config)

In [ ]:
def create_sprite(data):
    """
    Tile images into sprite image.
    Add any necessary padding
    """
    if len(data.shape) == 3:
        data = np.tile(data[..., np.newaxis], (1, 1, 1, 3))

    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, 0), (0, 0), (0, 0))
    data = np.pad(data, padding, mode="constant", constant_values=0)

    # Tile images into sprite
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3, 4))

    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    return data

In [ ]:
plot_to_projector(X, feature_vectors, y, ['Albert', 'Eline', 'Erik', 'Kevin', 'Marcel'])